In [2]:
!pip install pandas_profiling

  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27091 sha256=5726605bfa95c10c4c054bd673de6b018f6ca4937addd510425c2b44c66bbec9
  Stored in directory: c:\users\shubh\appdata\local\pip\cache\wheels\23\14\6e\4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
  Created wheel for phik: filename=phik-0.11.2-py3-none-any.whl size=1107420 sha256=d51515931e0f9f94ba162414e340491df5cd5b728b3cc8065c75abbb23a134ed
  Stored in directory: c:\users\shubh\appdata\local\pip\cache\wheels\72\9a\23\097c93fd8341220d92674891b6a9a30f0e389719d9b7291bd2
Successfully built htmlmin phik


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from pandas_profiling import ProfileReport

from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

import pickle

In [3]:
def reduce_range(x):
    # x: int (45, 105)
    # y: int (50, 100)
    # 100/105 = 0.9523
    return int(50 + 0.9523*(x-45))

In [4]:
def get_data(img_path, x=45, y=75):
    img = plt.imread(img_path)
    #img -> 150x150 input image
    output_pixel = img[45][75][0]
    if x==45:
        t = reduce_range(y)
        neighbours = [
            img[50][t-20][0],
            img[50][t-10][0],
            img[50][t][0],
            img[50][t+10][0],
            img[50][t+20][0]
        ]
    elif x==105:
        t = reduce_range(y)
        neighbours = [
            img[100][t-20][0],
            img[100][t-10][0],
            img[100][t][0],
            img[100][t+10][0],
            img[100][t+20][0]
        ]
    elif y==45:
        t = reduce_range(x)
        neighbours = [
            img[t-20][50][0],
            img[t-10][50][0],
            img[t][50][0],
            img[t+10][50][0],
            img[t+20][50][0],
        ]
    else:
        t = reduce_range(x)
        neighbours = [
            img[t-20][100][0],
            img[t-10][100][0],
            img[t][100][0],
            img[t+10][100][0],
            img[t+20][100][0]
        ]
    return neighbours, output_pixel

In [5]:
def get_df(dir, i=45, j=75):
    df = pd.DataFrame(columns=['input_1', 'input_2', 'input_3', 'input_4', 'input_5', 'output'])
    for fl in os.listdir(dir):
        row = get_data(dir+fl, i, j)
        df.loc[len(df.index)] = row[0] +[row[1]]
    return df

In [6]:
df = get_df('./Dataset/seg_test/seg_test/buildings/')
print(df)

    input_1 input_2 input_3 input_4 input_5 output
0       117     129      92     105      97    165
1        90      56      76      73      95    107
2       130     118     199     174     167    201
3       116     122     127     125     124    132
4         5     131     128     235     132     71
..      ...     ...     ...     ...     ...    ...
432     163     171     117     186     130     82
433     104     190     179     190      54     84
434     251     110     111     255     145     73
435     116     120      79     117      97    133
436     205     191     225     125      45    205

[437 rows x 6 columns]


In [7]:
profile = ProfileReport(df, title="Pandas Profiling Report")

In [8]:
profile.to_file("basic_report.html")

In [9]:
X = df[df.columns[:5]]
y = df['output']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
regr.score(X_test, y_test)

D:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


0.3336827084725126

In [12]:
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X, y)
neigh.score(X_test, y_test)

0.660606269586594

In [13]:
models = []
loc = [
    [45, 45],
    [45, 75],
    [45, 105],
    [75, 45],
    [75, 105],
    [105, 45],
    [105, 75],
    [105, 105]
]

In [14]:
for i,j in loc:
    print(i, j)
    df = get_df('./Dataset/seg_test/seg_test/buildings/', i, j)
    X = df[df.columns[:5]]
    y = df['output']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    models.append(KNeighborsRegressor(n_neighbors=2).fit(X, y))
    print(models[-1].score(X_test, y_test))

45 45
0.6298386158494249
45 75
0.660606269586594
45 105
0.6991415676781325
75 45
0.478104299198709
75 105
0.48800789093433916
105 45
0.5519831015781869
105 75
0.47938514515190755
105 105
0.5495442305166169


In [15]:
with open('knn_expansion_models.pkl','wb') as f:
     pickle.dump(models, f)